In [ ]:
training_job_name = 'profiler-gpu-2020-07-01-02-31-58-560'
region = 'us-east-1'

# Detect and analyze performance bottlenecks

This notebook will detect performance bottlenecks and perform a simple root cause analysis for provided sagemaker training job.

# 1. Install Dependencies


###  `smdebug`

The first thing you will need to do is install the private beta versions of the SageMaker Python SDK and the `smdebug` library. This will enable you to call a private version of the API that allows you to create SageMaker training jobs with the profiler enabled.

In [ ]:

! pip install -q ../smdebug-0.8.0b20200622-py3-none-any.whl


### Read profiler data: system metrics and framework metrics

Once the training job is running SageMaker will collect system and framework metrics. The following code cell is waiting for the system metrics & framework metrics to become available in S3. Once they are available you will be able to query and plot those metrics.

In [ ]:
import pandas as pd
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

# 2. Convert data into Pandas frame

## 2.1 Pandas frame for system metrics
 
Smdebug provides a class `PandasFrame` that converts profiler data into Pandas frames. 

In [ ]:
from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame

pf = PandasFrame(tj.profiler_s3_output_path)

Following function retrieves the latest events. If your training job is still running, you can periodically run this function to update the Pandas frames with the latest profiling data.

In [ ]:
system_metrics_df, framework_metrics_df = pf.get_latest_data()


Let's have a look on the Pandas frame for system metrics:

In [ ]:
system_metrics_df.head()

Pandas frame for framework metrics:

In [ ]:
framework_metrics_df.head()

# 3. Simple Analysis

## 3.1 Training job duration

First we check how long the job has been running:

In [ ]:
print('Training job run from:', min(system_metrics_df['timestamp']), 'to', max(system_metrics_df['timestamp']))

Get timestamp when initialization ended and training loop started:

In [ ]:
step_0 = framework_metrics_df[(framework_metrics_df['step'] == 0) 
                                           & (framework_metrics_df['framework_metric'] == 'Step:ModeKeys.TRAIN')]
training_loop_start = step_0['start_time_us'][1]

In [ ]:
print('Training loop started at ' + str(step_0['start_time']))

Last training/validation step:

In [ ]:
training_loop_end = max(framework_metrics_df[framework_metrics_df['framework_metric'].str.match('Step:ModeKeys.TRAIN')]['end_time_us'])

In [ ]:
print('Training loop took  ' + str(training_loop_end - training_loop_start) + 'us')

Create a Pie chart for training phases:

In [ ]:
finalization = max(system_metrics_df['timestamp_us']) - training_loop_end
training_job_phases = {'initialization': training_loop_start,
                       'training_loop': training_loop_end - training_loop_start, 
                       'finalization':  finalization if finalization > 0 else 0
                      }


In [ ]:
pd.Series(training_job_phases).plot.pie(figsize=(10, 10), 
                                     autopct='%.2f', 
                                     label='')

## 3.2 Low GPU utilization

Get all rows from Pandas frame which have GPU metrics.

In [ ]:
gpus = system_metrics_df[system_metrics_df['system_metric'].str.match('gpu')]
gpus.head()

Find all rows that have low GPU utilization:

In [ ]:
print(len(gpus.loc[gpus['value'] == 0.0]), "datapoints out of", len(gpus), "where GPU utilization was at 0.")

In [ ]:
print(len(gpus.loc[gpus['value'] < 50.0]), "datapoints out of", len(gpus), "where GPU utilization was below 50.")

Histogram of GPU utilization values:

In [ ]:
gpus['value'].plot.hist(bins=50, alpha=0.5)

Get GPU utilization values during training loop:

In [ ]:
gpus_training_loop = gpus[(gpus['timestamp_us'] > training_loop_start) 
                          & (gpus['timestamp_us'] < training_loop_end)]

In [ ]:
gpus_training_loop['value'].plot.hist(bins=50, alpha=0.5)

## 3.3 Low CPU utilization


Get all rows from Pandas frame which have CPU metrics.

In [ ]:
cpus = system_metrics_df[system_metrics_df['system_metric'].str.match('cpu')]
cpus.head()

Find all rows that have low CPU utilization:

In [ ]:
print(len(cpus.loc[cpus['value'] == 0.0]), "datapoints out of", len(cpus), "where CPU utilization was at 0.")

In [ ]:
print(len(cpus.loc[cpus['value'] < 50.0]), "datapoints out of", len(cpus), "where CPU utilization was below 50.")

Histogram of CPU utilization values:

In [ ]:
cpus['value'].plot.hist(bins=50, alpha=0.5)

Get CPU utilization during training loop

In [ ]:
cpus_training_loop = cpus[(cpus['timestamp_us'] > training_loop_start) 
                          & (cpus['timestamp_us'] < training_loop_end)]

In [ ]:
cpus_training_loop['value'].plot.hist(bins=50, alpha=0.5)

## 3.4 Detect CPU bottlenecks
The time periods where GPU was at 0 percent but CPU was at a 100 percent may indicate a CPU bottleneck. Let's retrieve those values:

In [ ]:
high_cpu = cpus[cpus['value'] > 90.0]
low_gpu = gpus[gpus['value'] <= 10.0]


Find rows where timestamp between both frames is the same:

In [ ]:
cpu_bottleneck = low_gpu.merge(high_cpu, on='timestamp', how='inner')
cpu_bottleneck = cpu_bottleneck.drop_duplicates(subset=['system_metric_x','timestamp'], keep='first')
cpu_bottleneck.head()


In [ ]:
for timestamp in cpu_bottleneck['timestamp'].unique():
    print('CPU bottleneck observed at', timestamp)
    

Get CPU bottlenecks that happen during the training loop:

In [ ]:
cpu_bottleneck_training_loop = cpu_bottleneck[cpu_bottleneck['timestamp_us_x'] > training_loop_start]

### 3.6 GPU usage - overview

Optimizing the performance of ML model training means to increase GPU usage. Let's create a Pie chart the shows the proportion between low and high GPU usage. We define the following intervals:
- GPU usage above 90%: optimal GPU usage
- GPU usage above 10% but below 90%: sub-optimal GPU usage
- GPU usage below 10% but CPU usage not at 100%: low GPU usage 
- CPU usage above 90% and GPU usage below 10%: low GPU usage likely due to CPU bottleneck
- initialization: GPU usage well be 0 because training loop has not started yet
- finalization: the time from end of training loop to end of training job

In [ ]:
system_usage = {'high_gpu_usage': len(gpus_training_loop[(gpus_training_loop['value'] > 90.0)]),
                'suboptimal_gpu_usage': len(gpus_training_loop[(gpus_training_loop['value'] < 90.0) & (gpus_training_loop['value'] > 10.0)]),
                'low_gpu_usage': len(gpus_training_loop[(gpus_training_loop['value'] <= 10.0)]) - len(cpu_bottleneck_training_loop),
                'cpu_bottleneck': len(cpu_bottleneck_training_loop),
                'initialization': len(gpus[gpus['timestamp_us'] < training_loop_start]),
                'finalization': len(gpus[gpus['timestamp_us'] > training_loop_end])}


In [ ]:
import matplotlib.pyplot as plt

pd.Series(system_usage).plot.pie(figsize=(7, 7), 
                                     autopct='%.2f', 
                                     label='')
plt.tight_layout()


## 3.5 Detect workload balancing issues in multi-GPU training

If usage one one GPU is much higher compared to the remaining GPUs, then this indicates an isuee with how workload is distributed across multiple GPUs. The following code cell will compute a histogram of usage values per GPU and measure the distance between the histograms.

In [ ]:
n_gpus = len(gpus['system_metric'].unique())

Compute histogram of usage values per GPU:

In [ ]:
import numpy as np

distributions = {}
for gpu in range(n_gpus):
    values = gpus[gpus['system_metric'] == 'gpu' + str(gpu)]['value']
    bins = np.arange(0, 100, 2)
    probs, binedges = np.histogram(values, bins=bins)
    distributions[gpu] = probs


Find which usage patterns exceed the threshold:

In [ ]:
import scipy

for gpu1 in range(n_gpus):
    for gpu2 in range(gpu1+1, n_gpus):
        m = (distributions[gpu1] + distributions[gpu2]) / 2
        divergence = (scipy.stats.entropy(distributions[gpu1], m) + scipy.stats.entropy(distributions[gpu2], m)) / 2
        distance = np.sqrt(divergence)
        
        if distance > 0.5:
            print('Workload between GPU', gpu1, 'and', gpu2, 'differs significantly. ')


## 3.6 Detect outliers in step duration

We would expect that the average duration of training steps should be roughly the same. If there are significant outliers, it may indicate an issue. The following code cell computes the step duration and its median value.

In [ ]:
train_steps = framework_metrics_df[framework_metrics_df['framework_metric']=='Step:ModeKeys.TRAIN']
step_durations = train_steps['end_time_us'] - train_steps['start_time_us']
median_train_steptime = step_durations.median()


Training step durations:

In [ ]:
step_durations.head()

Median of training step durations:

In [ ]:
median_train_steptime

Next we select all training steps where duration was larger than twice the median:

In [ ]:
outliers_train_steps = train_steps[step_durations > median_train_steptime * 2.0]
outliers_train_steps

Let's consider all values below this threshold as normal step duration:

In [ ]:
normal_train_steps = train_steps[step_durations < median_train_steptime * 2.0]

# 4. Root cause analysis

## 4.1. CPU bottlenecks
In this section, we will retrieve the function calls that were made at the time that a CPU bottleneck occured. We then compare the average time spent in those functions with the average time spent when there is no bottleneck. The difference between both may give an indication what the possible root cause is. 

For instance if the dataloading time significantly increases when GPU is at 0 and CPU is at 100 percent versus the dataloading time when GPU is at 100 percent then we can conclude that there is an issue in the data processing pipeline.


In [ ]:
cpu_bottleneck.head()

Helper function to compute cumulative time of a function for a certain timerange:

In [ ]:
def get_cumulative_time(unique_indexes):
    cumulative_time = {}
    training_phase = {}
    for index in unique_indexes:
        metric = framework_metrics_df['framework_metric'][index]
        if 'Step:ModeKeys' in metric:
            if metric not in training_phase:
                training_phase[metric] = 0
            training_phase[metric] += framework_metrics_df['end_time_us'][index] - framework_metrics_df['start_time_us'][index]
        elif metric not in cumulative_time:
            if metric not in cumulative_time:
                cumulative_time[metric] = 0
            cumulative_time[metric] += framework_metrics_df['end_time_us'][index] - framework_metrics_df['start_time_us'][index]
    return cumulative_time, training_phase

Iterate over the timestamps when a CPU bottleneck occured and find framework metrics for this timestep. 

In [ ]:
indexes = []
for timestamp in cpu_bottleneck['timestamp_us_x'].unique():
    inds = framework_metrics_df[(framework_metrics_df['start_time_us']  <= timestamp) 
                                & (framework_metrics_df['end_time_us'] >= timestamp)].index.tolist()
    if len(inds) > 0 :
        indexes.extend(inds)
unique_indexes = set(indexes)

Now that we have retrieved the indices of the relevant framework metrics, we sum metrics with the same name:

In [ ]:
cumulative_time, training_phase = get_cumulative_time(unique_indexes)
cumulative_time, training_phase

In [ ]:
from matplotlib import pyplot

pyplot.figure()
pyplot.subplot(122)
pyplot.subplots_adjust(wspace=1)
pd.Series(cumulative_time).plot.pie(figsize=(12, 12), 
                                     autopct='%.2f', 
                                     label='')
pyplot.subplot(121)
pd.Series(training_phase).plot.pie(figsize=(12, 12), 
                                     autopct='%.2f', 
                                     label='')

Normalize:

In [ ]:
for metric in cumulative_time:
    print("Average time spent in function", metric, ":", cumulative_time[metric]/len(unique_indexes), "us")
    
for metric in training_phase:
    print("Average time spent in function", metric, ":", training_phase[metric]/len(unique_indexes), "us")    

We have identified the functions that took most of the time when CPU bottlenecks occur. Let's compare the average time spent in those functions versus the average time spent in those functions when there is no bottleneck:

In [ ]:
high_gpu = gpus[gpus['value'] > 90.0]

In [ ]:
indexes = []
for timestamp in high_gpu['timestamp_us'].unique():
    inds = framework_metrics_df[(framework_metrics_df['start_time_us']  <= timestamp) 
                                & (framework_metrics_df['end_time_us'] >= timestamp)].index.tolist()
    if len(inds) > 0 :
        indexes.extend(inds)
unique_indexes = set(indexes)
cumulative_time, training_phase = get_cumulative_time(unique_indexes)
cumulative_time, training_phase

In [ ]:
from matplotlib import pyplot

pyplot.figure()
pyplot.subplot(122)
pyplot.subplots_adjust(wspace=1)
pd.Series(cumulative_time).plot.pie(figsize=(12, 12), 
                                     autopct='%.2f', 
                                     label='')
pyplot.subplot(121)
pd.Series(training_phase).plot.pie(figsize=(12, 12), 
                                     autopct='%.2f', 
                                     label='')

Normalize:

In [ ]:
for metric in cumulative_time:
    print("Average time spent in function", metric, ":", cumulative_time[metric]/len(unique_indexes), "us")

for metric in training_phase:
    print("Average time spent in function", metric, ":", training_phase[metric]/len(unique_indexes), "us") 

### Outliers in step duration

In [ ]:
function_calls = framework_metrics_df[(framework_metrics_df["framework_metric"] != 'Step:ModeKeys.TRAIN') 
                                      & (framework_metrics_df["framework_metric"] != 'Step:ModeKeys.EVAL')]


In [ ]:
function_calls.head()

In [ ]:
indexes = []
for start_time, end_time in zip(outliers_train_steps['start_time_us'].tolist(), 
                                outliers_train_steps['end_time_us'].tolist()):
    inds = function_calls[(function_calls['end_time_us']  >= start_time) 
                        & (function_calls['start_time_us'] <= end_time)].index.tolist()
    if len(inds) > 0 :
        indexes.extend(inds)
unique_indexes = set(indexes)
cumulative_time, _ = get_cumulative_time(unique_indexes)
cumulative_time

In [ ]:
pd.Series(cumulative_time).plot.pie(figsize=(6, 6), 
                                     autopct='%.2f', 
                                     label='')

In [ ]:
avg_time = {}
avg_time['cpu_bottleneck'] = {}
for metric in cumulative_time:
    avg_time['cpu_bottleneck'][metric] =  cumulative_time[metric]/len(unique_indexes)
    print("Average time spent in function", metric, ":", cumulative_time[metric]/len(unique_indexes), "us")

Let's compare the average time spent in those functions versus the average time spent in those functions when train steps have a 'normal' duration:

In [ ]:
indexes = []
for start_time, end_time in zip(normal_train_steps['start_time_us'].tolist(), 
                                normal_train_steps['end_time_us'].tolist()):
    inds = function_calls[(function_calls['end_time_us']  >= start_time) 
                        & (function_calls['start_time_us'] <= end_time)].index.tolist()
    if len(inds) > 0 :
        indexes.extend(inds)
unique_indexes = set(indexes)
cumulative_time, _ = get_cumulative_time(unique_indexes)
cumulative_time

In [ ]:
pd.Series(cumulative_time).plot.pie(figsize=(6, 6), 
                                     autopct='%.2f', 
                                     label='')

In [ ]:
avg_time['normal'] = {}
for metric in cumulative_time:
    avg_time['normal'][metric] =  cumulative_time[metric]/len(unique_indexes)
    print("Average time spent in function", metric, ":", cumulative_time[metric]/len(unique_indexes), "us")

Compare average time spent in those functions when there is a CP bottlenck versus no bottleneck:

In [ ]:
avg_time

In [ ]:
pd.DataFrame(avg_time).plot(kind='bar', subplots=True)

Get CPU/GPU utilization during train step outliers. If utilization is low, it may indicate a system stall. This can for instance happen if a process is waiting for an I/O request being resolved.

In [ ]:
indexes = []
for start_time, end_time in zip(outliers_train_steps['start_time_us'].tolist(), 
                                outliers_train_steps['end_time_us'].tolist()):
    inds = system_metrics_df[(system_metrics_df['timestamp_us']  >= start_time) 
                        & (system_metrics_df['timestamp_us'] <= end_time)].index.tolist()
    if len(inds) > 0 :
        indexes.extend(inds)
unique_indexes = set(indexes)

In [ ]:
system_metrics_train_step_outliers = system_metrics_df.iloc[indexes,:]
gpus = system_metrics_train_step_outliers[system_metrics_train_step_outliers['system_metric'].str.match('gpu')]
cpus = system_metrics_train_step_outliers[system_metrics_train_step_outliers['system_metric'].str.match('cpu')]
gpus['value'].plot.hist(bins=50, alpha=0.5, legend=True, label="GPUs")
cpus['value'].plot.hist(bins=50, alpha=0.5, legend=True, label="CPUs")

For comparison CPU/GPU utilization during when training steps have normal duration:

In [ ]:
indexes = []
for start_time, end_time in zip(normal_train_steps['start_time_us'].tolist(), 
                                normal_train_steps['end_time_us'].tolist()):
    inds = system_metrics_df[(system_metrics_df['timestamp_us']  >= start_time) 
                        & (system_metrics_df['timestamp_us'] <= end_time)].index.tolist()
    if len(inds) > 0 :
        indexes.extend(inds)
unique_indexes = set(indexes)

In [ ]:
system_metrics_normal_train_step = system_metrics_df.iloc[indexes,:]
gpus = system_metrics_normal_train_step[system_metrics_normal_train_step['system_metric'].str.match('gpu')]
cpus = system_metrics_normal_train_step[system_metrics_normal_train_step['system_metric'].str.match('cpu')]
gpus['value'].plot.hist(bins=50, alpha=0.5, legend=True, label="GPUs")
cpus['value'].plot.hist(bins=50, alpha=0.5, legend=True, label="CPUs")